In [1]:
!pip install datasets
!pip install peft transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 686.4 kB

In [2]:
from datasets import load_dataset
import pandas as pd

In [3]:
# Load train dataset
ds = load_dataset("ailsntua/QEvasion")

# Convert to pandas and keep only useful columns
df_train = ds["train"].to_pandas()[["question","interview_answer",
                                   "label","annotator_id"]]

# Create dictionary with key as the annotator and value the Dataframe with
# only the corresponding sQAs
split_dfs = {
    category: group[["question","interview_answer", "label"]]
    for category, group in df_train.groupby('annotator_id')
    }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/779 [00:00<?, ?B/s]

QEvasion.csv:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

In [4]:
temp_train_set = split_dfs[85]
temp_train_set

,question,interview_answer,label
0,How would you respond to the accusation that t...,"\nThe President. Well, look, first of all, the...",1.1 Explicit
1,Do you think President Xi is being sincere abo...,"\nThe President. Well, look, first of all, the...",2.4 General
2,1. Q1: Do you believe the country's slowdown a...,"\nThe President. Look, I think China has a dif...",2.3 Partial/half-answer
3,2. Q2: Are you worried about the meeting betwe...,"\nThe President. Look, I think China has a dif...",2.1 Dodging
4,Is the President's engagement with Asian coun...,"\nThe President. Well, I hope I get to see Mr....",1.1 Explicit
...,...,...,...
3064,Is Syria part of the talks?,\nThe President. Proliferation is a part of th...,2.2 Deflection
3065,Is Syria included in something?,"\nThe President. Elaine [Elaine Quijano, Cable...",2.6 Declining to answer
3170,Can I have a followup?,"\nThe President. Yes, you can, because you're ...",1.1 Explicit
3192,Deadline for the Kosovo independence?,\nPresident Bush. A decline?,2.8 Clarification


In [6]:
temp_train_set["label"].unique()

array(['1.1 Explicit', '2.4 General', '2.3 Partial/half-answer',
       '2.1 Dodging', '1.2 Implicit', '2.2 Deflection',
       '2.6 Declining to answer', '2.7 Claims ignorance',
       '2.8 Clarification'], dtype=object)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from peft import get_peft_model, LoraConfig, TaskType

# Load pre-trained Roberta model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,     # Task type: sequence classification
    r=8,                            # Rank of the low-rank adaptation
    lora_alpha=16,                  # Scaling factor
    lora_dropout=0.1,               # Dropout probability for LoRA layers
    target_modules=["query", "value"] # Target modules to inject LoRA into (attention projections)
)

# Add LoRA to the model
model = get_peft_model(model, lora_config)

# Freeze the original model parameters (LoRA will only train the adapters)
for param in model.base_model.parameters():
    param.requires_grad = False

# Define the dataset and data loader
# Assuming the dataset returns a dictionary with 'input_ids', 'attention_mask', and 'labels'
dataset = ...  # Your dataset here
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Move model to the appropriate device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(5):  # adjust the number of epochs as needed
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')

    model.eval()

# Save the model with LoRA adapters
model.save_pretrained("roberta-lora")